<a href="https://colab.research.google.com/github/2pterons/twigfarm/blob/main/eomiso_StyleTransfer_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/eomiso/NLP_text-style-transfer

Cloning into 'NLP_text-style-transfer'...
remote: Enumerating objects: 605, done.
remote: Counting objects: 100% (605/605), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 605 (delta 358), reused 598 (delta 355), pack-reused 0
Receiving objects: 100% (605/605), 37.00 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [ ]:
!pwd

/content


In [ ]:
%cd /content/NLP_text-style-transfer

/content/NLP_text-style-transfer


In [ ]:
!pip install transformers

In [ ]:
!python transfer.py --ckpt_path ../content --clf_ckpt_path ../content --dataset yelp

------------------------------------------------
{   'batch_size': 64,
    'ckpt_path': '../content',
    'clf_ckpt_path': 'checkpoints/yelp_clf.pt',
    'cuda_device': 0,
    'dataset': 'yelp',
    'device': device(type='cpu'),
    'dim_emb': 768,
    'dim_y': 200,
    'dim_z': 500,
    'disc_lr': 5e-05,
    'epochs': 20,
    'filter_sizes': [1, 2, 3, 4, 5],
    'gan_type': 'vanilla',
    'gp_weight': 1.0,
    'language': 'en',
    'load_ckpt': 'ckpts/wgan_no_threshold.pt',
    'log_interval': 100,
    'lr': 0.0005,
    'max_grad_norm': 1.0,
    'max_seq_length': 64,
    'mode': 'train',
    'n_filters': 128,
    'n_samples': 1000,
    'num_workers': 4,
    'rho': 1,
    'second_stage_num': 2,
    'temperature': 0.1,
    'test_text_path': 'data/yelp/yelp.sentiment.test',
    'text_file_path': 'data/yelp/yelp.sentiment.train',
    'threshold': None,
    'transfer_max_len': 64,
    'transfer_result_save_path': None,
    'transfer_to': 1,
    'two_stage': False,
    'use_gumbel': True,
 

In [ ]:
!python transfer.py --train ../data/yelp/yelp.sentiment.train --dev ../data/yelp/sentiment.dev --output ../tmp/sentiment.dev --vocab ../tmp/yelp.vocab --model ../tmp/model

usage: transfer.py [-h] --ckpt_path CKPT_PATH [--clf_ckpt_path CLF_CKPT_PATH]
                   [--dataset {yelp,nsmc}] [--text_file_path TEXT_FILE_PATH]
                   [--val_text_file_path VAL_TEXT_FILE_PATH]
                   [--batch_size BATCH_SIZE] [--max_seq_length MAX_SEQ_LENGTH]
                   [--num_workers NUM_WORKERS] [--dim_y DIM_Y] [--dim_z DIM_Z]
                   [--dim_emb DIM_EMB]
                   [--filter_sizes FILTER_SIZES [FILTER_SIZES ...]]
                   [--n_filters N_FILTERS] [--epochs EPOCHS]
                   [--weight_decay WEIGHT_DECAY]
                   [--max_grad_norm MAX_GRAD_NORM] [--lr LR]
                   [--disc_lr DISC_LR] [--temperature TEMPERATURE]
                   [--use_gumbel USE_GUMBEL] [--rho RHO]
                   [--two_stage TWO_STAGE]
                   [--second_stage_num SECOND_STAGE_NUM]
                   [--gan_type {vanilla,wgan-gp,lsgan}]
                   [--gp_weight GP_WEIGHT] [--log_interval LOG_INTER

In [ ]:
from tqdm import tqdm
import torch

from bert_pretrained import bert_tokenizer
from options import args


def style_transfer(encoder=None, generator=None, text_path=None, n_samples=100):
    # save result if path is given
    if args.transfer_result_save_path is not None:
        fw = open(args.transfer_result_save_path, 'a')
    else:
        fw = None

    # interactive mode
    if text_path is None:
        if fw is not None:
            fw.write('\n' + "=" * 50 + '\n')
            fw.write("Interactive transfer from {} -> {}\n".format(
                str(1 - args.transfer_to),
                str(args.transfer_to)
            ))
            fw.write("=" * 50 + '\n')
        try:
            while True:
                fmt = "Enter text to transfer to style {} (Ctrl+C to exit): "
                text = input(fmt.format(args.transfer_to))
                tokens = bert_tokenizer.encode(text, add_special_tokens=False)
                tokens = (
                    [bert_tokenizer.bos_token_id]
                    + tokens
                    + [bert_tokenizer.eos_token_id]
                )
                tokens = torch.LongTensor([tokens]).transpose(0, 1)
                original_label = torch.FloatTensor([1 - args.transfer_to])
                output = generate_text(
                    encoder.to(args.device),
                    generator.to(args.device),
                    original_label.to(args.device),
                    tokens.to(args.device)
                )
                print("Transfer result:", output)
                if fw is not None:
                    fw.write(text + ' -> ' + output + '\n')

        except KeyboardInterrupt:
            if fw is not None:
                fw.close()
            print("\nEnd interactive transfer\n")

    # load data from text path
    else:
        if fw is not None:
            fw.write('\n' + "=" * 50 + '\n')
            fw.write("Transfer from file: {}\n".format(text_path))
            fw.write("Number of samples: {}\n".format(n_samples))
            fw.write("=" * 50 + '\n')

        pbar = tqdm(total=n_samples)
        counter = 0
        inputs0, inputs1 = [], []
        outputs0, outputs1 = [], []
        with open(text_path, 'r') as text_file:
            for line in text_file:
                counter += 1
                if counter == 1:
                    continue
                _, text, label = line.strip().split('\t')
                tokens = bert_tokenizer.encode(text, add_special_tokens=False)
                tokens = (
                    [bert_tokenizer.bos_token_id]
                    + tokens
                    + [bert_tokenizer.eos_token_id]
                )
                tokens = torch.LongTensor([tokens]).transpose(0, 1)
                original_label = torch.FloatTensor([int(label)])
                output = generate_text(
                    encoder.to(args.device),
                    generator.to(args.device),
                    original_label.to(args.device),
                    tokens.to(args.device)
                )
                if int(label) == 0:
                    inputs0.append(text)
                    outputs0.append(output)
                else:
                    inputs1.append(text)
                    outputs1.append(output)
                pbar.update()
                if fw is not None:
                    fw.write(label + ' > ' + str(1-int(label)) + ': '+ text + ' -> ' + output + '\n')
                if counter > n_samples:
                    break

        if fw is not None:
            fw.close()
        return inputs0, inputs1, outputs0, outputs1


def generate_text(encoder, generator, original_label, tokens):
    src_len = [len(tokens)]
    predictions = generator.transfer(
        encoder(original_label, tokens, src_len),  # hidden state
        1 - original_label,  # transfer label
        eos_token_id=bert_tokenizer.eos_token_id,
        max_len=args.transfer_max_len
    )
    # change this part to first occurence of ber_tokenizer.eos_token_id
    #if predictions[-1] == bert_tokenizer.eos_token_id:
    #    predictions = predictions[:-1]

    try:
        eos_idx = predictions.index(bert_tokenizer.eos_token_id)
        predictions = predictions[:eos_idx]
    except ValueError:
        predictions=predictions[:-1]
    return bert_tokenizer.decode(predictions)


# def _transfer():
#     device = torch.device('cuda:{}'.format(args.cuda_device) if torch.cuda.is_available() else 'cpu')
    
#     # 1. get model
#     embedding = get_bert_word_embedding().to(device).eval()
#     encoder = Encoder(embedding, args.dim_y, args.dim_z).to(device).eval()
#     generator = Generator(embedding, args.dim_y, args.dim_z, args.temperature, bert_tokenizer.bos_token_id, use_gumbel=args.use_gumbel).to(device).eval()
    
#     # 2. load checkpoint
#     ckpt = torch.load(args.ckpt_path, map_location=device)
#     embedding.load_state_dict(ckpt['embedding_state_dict'])
#     encoder.load_state_dict(ckpt['encoder_state_dict'])
#     generator.load_state_dict(ckpt['generator_state_dict'])
    
#     # 3. transfer!
#     if args.transfer_result_save_path is not None:
#         fw = open(args.transfer_result_save_path, 'w')
#     else:
#         fw = None
            
#     if args.test_text_path is None:
#         # interactive mode
#         while True:
#             text = input("Enter text to transfer to stye {} (Ctrl+C to exit): ".format(args.transfer_to))
#             text_tokens = [bert_tokenizer.bos_token_id] + bert_tokenizer.encode(text, add_special_tokens=False) + [bert_tokenizer.eos_token_id]
#             text_tokens_tensor = torch.LongTensor([text_tokens]).transpose(0, 1).to(device)
#             src_len = [len(text_tokens)]
#             original_label = torch.FloatTensor([1-args.transfer_to]).to(device)
#             transfer_label = torch.FloatTensor([args.transfer_to]).to(device)
            
#             z = encoder(original_label, text_tokens_tensor, src_len)
#             predictions = generator.transfer(z, transfer_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
#             if predictions[-1] == bert_tokenizer.eos_token_id:
#                 predictions = predictions[:-1]
                
#             result = bert_tokenizer.decode(predictions)
#             print("Transfer Result:", result)
#             if fw is not None:
#                 fw.write(text + ' -> ' + result + '\n')
                
#             if args.test_recon:
#                 recon = generator.transfer(z, original_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
#                 if recon[-1] == bert_tokenizer.eos_token_id:
#                     recon = recon[:-1]
#                 print("Recon:", bert_tokenizer.decode(recon))
            
#     else:

#         for line in args.test_text_path:
#             line = line.strip()
#             text = line
#             text_tokens = [bert_tokenizer.bos_token_id] + bert_tokenizer.encode(text, add_special_tokens=False) + [bert_tokenizer.eos_token_id]
#             text_tokens_tensor = torch.LongTensor([text_tokens]).transpose(0, 1).to(device)
#             src_len = [len(text_tokens)]
#             original_label = torch.FloatTensor([1-args.transfer_to]).to(device)
#             transfer_label = torch.FloatTensor([args.transfer_to]).to(device)
            
#             z = encoder(original_label, text_tokens_tensor, src_len)
#             predictions = generator.transfer(z, transfer_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
#             if predictions[-1] == bert_tokenizer.eos_token_id:
#                 predictions = predictions[:-1]
                
#             result = bert_tokenizer.decode(predictions)
#             print("Transfer Result:", result)
#             if fw is not None:
#                 fw.write(text + ' -> ' + result + '\n')
                
#             if args.test_recon:
#                 recon = generator.transfer(z, original_label, eos_token_id=bert_tokenizer.eos_token_id, max_len=args.transfer_max_len)
#                 if recon[-1] == bert_tokenizer.eos_token_id:
#                     recon = recon[:-1]
#                 print("Recon:", bert_tokenizer.decode(recon))
            
# if __name__ == '__main__':
#     transfer()

usage: /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py
       [-h] --ckpt_path CKPT_PATH [--clf_ckpt_path CLF_CKPT_PATH]
       [--dataset {yelp,nsmc}] [--text_file_path TEXT_FILE_PATH]
       [--val_text_file_path VAL_TEXT_FILE_PATH] [--batch_size BATCH_SIZE]
       [--max_seq_length MAX_SEQ_LENGTH] [--num_workers NUM_WORKERS]
       [--dim_y DIM_Y] [--dim_z DIM_Z] [--dim_emb DIM_EMB]
       [--filter_sizes FILTER_SIZES [FILTER_SIZES ...]]
       [--n_filters N_FILTERS] [--epochs EPOCHS] [--weight_decay WEIGHT_DECAY]
       [--max_grad_norm MAX_GRAD_NORM] [--lr LR] [--disc_lr DISC_LR]
       [--temperature TEMPERATURE] [--use_gumbel USE_GUMBEL] [--rho RHO]
       [--two_stage TWO_STAGE] [--second_stage_num SECOND_STAGE_NUM]
       [--gan_type {vanilla,wgan-gp,lsgan}] [--gp_weight GP_WEIGHT]
       [--log_interval LOG_INTERVAL] [--language {ko,en}]
       [--threshold THRESHOLD] [--mode {train,transfer,interactive}]
       [--test_text_path TEST_TEXT_PATH] [--transfer_to {

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
